# Импорт библиотек

In [ ]:
!pip install kmeans_pytorch
!pip install recbole
!pip install ray

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 MB 11.2 MB/s eta 0:00:00


In [ ]:
import ast
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle
import torch

import warnings
warnings.filterwarnings('ignore')

from collections import Counter
from random import randint, random
from scipy.sparse import coo_matrix, hstack
from sklearn.metrics.pairwise import euclidean_distances, cosine_distances, cosine_similarity


In [ ]:
import time

In [ ]:
import logging
from logging import getLogger
from recbole.config import Config
from recbole.data import create_dataset, data_preparation
from recbole.model.sequential_recommender import GRU4Rec, Caser
from recbole.trainer import Trainer
from recbole.utils import init_seed, init_logger
from recbole.quick_start import run_recbole
from recbole.model.general_recommender import RecVAE

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import warnings
warnings.filterwarnings('ignore')

# Чтение данных

In [ ]:
interactions_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ИТМО/Recsys/5/interactions_processed_kion.csv')
users_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ИТМО/Recsys/5/users_processed_kion.csv')
items_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ИТМО/Recsys/5/items_processed_kion.csv')

In [ ]:
interactions_df['t_dat'] = pd.to_datetime(interactions_df['last_watch_dt'], format="%Y-%m-%d")
interactions_df['timestamp'] = interactions_df.t_dat.values.astype(np.int64) // 10 ** 9

In [ ]:
df = interactions_df[['user_id', 'item_id', 'timestamp']].rename(
    columns={'user_id': 'user_id:token', 'item_id': 'item_id:token', 'timestamp': 'timestamp:float'})

In [ ]:
!mkdir recbox_data

In [ ]:
df.to_csv('recbox_data/recbox_data.inter', index=False, sep='\t')

# Подготовка данных и модели

In [ ]:
parameter_dict = {
    'data_path': '',
    'USER_ID_FIELD': 'user_id',
    'ITEM_ID_FIELD': 'item_id',
    'TIME_FIELD': 'timestamp',
    'device': 'GPU',
    'user_inter_num_interval': "[40,inf)",
    'item_inter_num_interval': "[40,inf)",
    'load_col': {'inter': ['user_id', 'item_id', 'timestamp']},
    'neg_sampling': None,
    'epochs': 10,
    'eval_args': {
        'split': {'RS': [9, 0, 1]},
        'group_by': 'user',
        'order': 'TO',
        'mode': 'full'}
}
config = Config(model='MultiVAE', dataset='recbox_data', config_dict=parameter_dict)

# init random seed
init_seed(config['seed'], config['reproducibility'])

# logger initialization
init_logger(config)
logger = getLogger()
# Create handlers
c_handler = logging.StreamHandler()
c_handler.setLevel(logging.INFO)
logger.addHandler(c_handler)

# write config info into log
# logger.info(config)

In [ ]:
dataset = create_dataset(config)
logger.info(dataset)

In [ ]:
# dataset splitting
train_data, valid_data, test_data = data_preparation(config, dataset)

# Обучение моделей

In [ ]:
%%time
model_list = ['MultiVAE', 'MultiDAE', 'MacridVAE',
              "NeuMF", "RecVAE"]
results = {}

for model_name in model_list:
    print(f"running {model_name}...")
    start = time.time()
    result = run_recbole(model=model_name, dataset = 'recbox_data',config_dict = parameter_dict)
    results[model_name] = result
    t = time.time() - start
    print(f"It took {t/60:.2f} mins")
    print(result)

running MultiVAE...


command line args [-f /root/.local/share/jupyter/runtime/kernel-721ec768-09a6-4b54-bf44-4e614324ec3c.json] will not be used in RecBole
Max value of user's history interaction records has reached 20.9471766848816% of the total.
Evaluate   : 100%|███████████████████| 13354/13354 [02:17<00:00, 97.10it/s, GPU RAM: 0.38 G/14.75 G]


It took 3.47 mins
{'best_valid_score': -inf, 'valid_score_bigger': True, 'best_valid_result': None, 'test_result': OrderedDict([('recall@10', 0.0834), ('mrr@10', 0.1671), ('ndcg@10', 0.0816), ('hit@10', 0.3466), ('precision@10', 0.0462)])}
running MultiDAE...


command line args [-f /root/.local/share/jupyter/runtime/kernel-721ec768-09a6-4b54-bf44-4e614324ec3c.json] will not be used in RecBole
Max value of user's history interaction records has reached 20.9471766848816% of the total.
Evaluate   : 100%|███████████████████| 13354/13354 [02:29<00:00, 89.20it/s, GPU RAM: 0.38 G/14.75 G]


It took 4.38 mins
{'best_valid_score': -inf, 'valid_score_bigger': True, 'best_valid_result': None, 'test_result': OrderedDict([('recall@10', 0.0837), ('mrr@10', 0.1657), ('ndcg@10', 0.0814), ('hit@10', 0.3466), ('precision@10', 0.0463)])}
running MacridVAE...


...

It took 9.35 mins
{'best_valid_score': -inf, 'valid_score_bigger': True, 'best_valid_result': None, 'test_result': OrderedDict([('recall@10', 0.0827), ('mrr@10', 0.1548), ('ndcg@10', 0.0775), ('hit@10', 0.3469), ('precision@10', 0.0455)])}
running NeuMF...


command line args [-f /root/.local/share/jupyter/runtime/kernel-721ec768-09a6-4b54-bf44-4e614324ec3c.json] will not be used in RecBole
Evaluate   : 100%|███████████████████| 13354/13354 [03:36<00:00, 61.79it/s, GPU RAM: 0.95 G/14.75 G]


It took 10.44 mins
{'best_valid_score': -inf, 'valid_score_bigger': True, 'best_valid_result': None, 'test_result': OrderedDict([('recall@10', 0.0687), ('mrr@10', 0.1181), ('ndcg@10', 0.0607), ('hit@10', 0.3008), ('precision@10', 0.038)])}
running RecVAE...


command line args [-f /root/.local/share/jupyter/runtime/kernel-721ec768-09a6-4b54-bf44-4e614324ec3c.json] will not be used in RecBole
Max value of user's history interaction records has reached 20.9471766848816% of the total.
Evaluate   : 100%|███████████████████| 13354/13354 [04:24<00:00, 50.43it/s, GPU RAM: 0.95 G/14.75 G]


It took 7.13 mins
{'best_valid_score': -inf, 'valid_score_bigger': True, 'best_valid_result': None, 'test_result': OrderedDict([('recall@10', 0.0846), ('mrr@10', 0.1661), ('ndcg@10', 0.0818), ('hit@10', 0.3523), ('precision@10', 0.0469)])}
CPU times: user 29min 50s, sys: 1min 52s, total: 31min 43s
Wall time: 34min 46s


# Результаты

In [ ]:
results

{'MultiVAE': {'best_valid_score': -inf,
  'valid_score_bigger': True,
  'best_valid_result': None,
  'test_result': OrderedDict([('recall@10', 0.0834),
               ('mrr@10', 0.1671),
               ('ndcg@10', 0.0816),
               ('hit@10', 0.3466),
               ('precision@10', 0.0462)])},
 'MultiDAE': {'best_valid_score': -inf,
  'valid_score_bigger': True,
  'best_valid_result': None,
  'test_result': OrderedDict([('recall@10', 0.0837),
               ('mrr@10', 0.1657),
               ('ndcg@10', 0.0814),
               ('hit@10', 0.3466),
               ('precision@10', 0.0463)])},
 'MacridVAE': {'best_valid_score': -inf,
  'valid_score_bigger': True,
  'best_valid_result': None,
  'test_result': OrderedDict([('recall@10', 0.0827),
               ('mrr@10', 0.1548),
               ('ndcg@10', 0.0775),
               ('hit@10', 0.3469),
               ('precision@10', 0.0455)])},
 'NeuMF': {'best_valid_score': -inf,
  'valid_score_bigger': True,
  'best_valid_result': Non

Как мы видим, самые высокие показатели у RecVAE

# Рекомендации

In [ ]:
config = Config(model='RecVAE', dataset='recbox_data', config_dict=parameter_dict)

model = RecVAE(config, dataset)

loaded_model = torch.load("/content/saved/RecVAE-Dec-12-2023_12-29-45.pth")
model.load_state_dict(loaded_model["state_dict"]).to("cuda")

In [ ]:
def recommend_items(user_id, dataset, model, k_recos):
    if (user_id in dataset.field2token_id[dataset.uid_field] and user_id != "[PAD]"):
        model.eval()
        user = np.isin(dataset[dataset.uid_field].numpy(), dataset.token2id(dataset.uid_field, [user_id]))
        dataset_user = dataset[user].to("cuda")
        predict = model.full_sort_predict(dataset_user).view(-1, test_data.dataset.item_num)
        predict[:, 0] = -np.inf
        item_indices = torch.topk(predict, k_recos).indices[0].to_list()
        recos = dataset.id2token(dataset.iid_field, [item_indices]).to_list()
    else:
        recos = []
    return recos

recos_data = []
users = dataset.field2token_id[dataset.uid_field]

counter = 0
for user_id in users:
    counter+=1
    recos_for_user = recommend_items(user_id, dataset, model,10)
    if recos_for_user:
        recos_data.extend([(user_id, item_id) for item_id in recos_for_user])
    if counter % 100 == 0:
        print(f"Counter value: {counter}")

recos_df = pd.DataFrame(recos_data, columns=["user_id", "item_id"])

Counter value: 100
Counter value: 200
Counter value: 300
Counter value: 400
Counter value: 500
Counter value: 600
Counter value: 700
Counter value: 800
Counter value: 900
Counter value: 1000
Counter value: 1100
Counter value: 1200
Counter value: 1300
Counter value: 1400
Counter value: 1500
Counter value: 1600
Counter value: 1700
Counter value: 1800
Counter value: 1900
Counter value: 2000
Counter value: 2100
Counter value: 2200
Counter value: 2300
Counter value: 2400
Counter value: 2500
Counter value: 2600
Counter value: 2700
Counter value: 2800
Counter value: 2900
Counter value: 3000
Counter value: 3100
Counter value: 3200
Counter value: 3300
Counter value: 3400
Counter value: 3500
Counter value: 3600
Counter value: 3700
Counter value: 3800
Counter value: 3900
Counter value: 4000
Counter value: 4100
Counter value: 4200
Counter value: 4300
Counter value: 4400
Counter value: 4500
Counter value: 4600
Counter value: 4700
Counter value: 4800
Counter value: 4900
Counter value: 5000
Counter v

In [ ]:
recos_df

,user_id,item_id
0,176549,7626
0,176549,12173
0,176549,14317
0,176549,11310
0,176549,9728
...,...,...
13353,329210,13243
13353,329210,7582
13353,329210,11985
13353,329210,10761


In [ ]:
recos_df.to_csv('/content/drive/MyDrive/Colab Notebooks/ИТМО/Recsys/recbole_recos.csv')

# Что сделано

1. Проведены эксперименты из фреймворка Recbole

2. Выбрана лучшая модель и интегрирована в сервис